<a href="https://colab.research.google.com/github/Eunchae-L/Dysarthria-Whisper-Finetuning/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [ ]:
!pip install -q datasets librosa evaluate jiwer gradio bitsandbytes==0.37 accelerate
!pip install -q git+https://github.com/huggingface/peft.git@main
!pip install torchaudio
!pip install --upgrade bitsandbytes
!pip install --upgrade peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/6

In [ ]:
import torch

test_dysarthria_data = torch.load("test_dysarthria.pt")
test_nondysarthria_data = torch.load("test_non_dysarthria.pt")

<ipython-input-3-50c6224c4eea>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_dysarthria_data = torch.load("test_dysarthria.pt")
<ipython-input-3-50c6224c4eea>:4: Fut

In [ ]:
from datasets import Dataset, DatasetDict

# 여러 스플릿으로 관리하기 위한 DatasetDict 생성
dataset_dict = DatasetDict({
    'dysarthria': Dataset.from_list(test_dysarthria_data),   # train split
    'non_dysarthria': Dataset.from_list(test_nondysarthria_data),  # validation split
})

In [ ]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer
import torch
from evaluate import load

model_name = "openai/whisper-small"
model = WhisperForConditionalGeneration.from_pretrained(model_name).to("cuda")

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="en", task="transcribe")
processor = WhisperProcessor.from_pretrained(model_name, language="en", task="transcribe")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
data = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["dysarthria"])

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    dysarthria: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 257
    })
    non_dysarthria: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 257
    })
})

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Dysarthria Test

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

# 커스텀 WER 계산 함수
def custom_wer(reference, hypothesis):
    """
    긴 예측 오류 또는 빈 예측 오류를 WER=1로 설정.
    """
    if not hypothesis or len(hypothesis.split()) > 2 * len(reference.split()):
        return 1.0  # WER = 1로 처리
    else:
        return metric.compute(predictions=[hypothesis], references=[reference])

# 데이터 로더 및 설정
dysarthria_data = DataLoader(data["dysarthria"], batch_size=8, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
normalizer = BasicTextNormalizer()

# 평가 결과 저장 변수
predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(dysarthria_data)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            # 예측 생성
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            # 라벨 처리
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

            # 결과 저장
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])

        # 메모리 정리
        del generated_tokens, labels, batch
    gc.collect()

# 커스텀 WER 계산
wer_values = [custom_wer(ref, pred) for ref, pred in zip(references, predictions)]
normalized_wer_values = [custom_wer(ref, pred) for ref, pred in zip(normalized_references, normalized_predictions)]

# 평균 WER 계산
wer = 100 * np.mean(wer_values)
normalized_wer = 100 * np.mean(normalized_wer_values)

eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

# 결과 출력
print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

  0%|          | 0/33 [00:00<?, ?it/s]<ipython-input-21-d99269d36d82>:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 33/33 [00:56<00:00,  1.72s/it]


wer=79.86935243744581 and normalized_wer=53.93864458066792
{'eval/wer': 79.86935243744581, 'eval/normalized_wer': 53.93864458066792}


In [ ]:
for idx, (pred, ref) in enumerate(zip(predictions, references)):
    print(f"Index {idx}:\nPrediction: {pred}\nReference: {ref}\n")

Index 0:
Prediction:  Quebec
Reference: quebec

Index 1:
Prediction:  It also provides for friends to clear slumps and help colleges build dormitories.
Reference: It also provides for funds to clear slums and help colleges build dormitories.

Index 2:
Prediction:  Done.
Reference: darn

Index 3:
Prediction:  The job provides many benefits.
Reference: The job provides many benefits.

Index 4:
Prediction:  Bug
Reference: bug

Index 5:
Prediction:  We have an excellent team.
Reference: We have an excellent team.

Index 6:
Prediction:  Juliet.
Reference: juliet

Index 7:
Prediction:  Go
Reference: grow

Index 8:
Prediction:  If you destroy confidence in banks, you do something to the economy, he said.
Reference: If you destroy confidence in banks, you do something to the economy, he said.

Index 9:
Prediction:  Yeah.
Reference: the

Index 10:
Prediction:  Spain
Reference: spain

Index 11:
Prediction:  World!
Reference: warm

Index 12:
Prediction:  End
Reference: and

Index 13:
Prediction: 

In [ ]:
def compare_lists(predictions, references):
    # Initialize counters
    exact_matches = 0
    mismatches = 0

    for pred, ref in zip(predictions, references):
        # Remove spaces for comparison
        pred_clean = pred.replace(" ", "")
        ref_clean = ref.replace(" ", "")

        if pred_clean == ref_clean:
            exact_matches += 1
        else:
            mismatches += 1

    return exact_matches, mismatches


# Compare the lists and get results
exact_matches, mismatches = compare_lists(predictions, references)

# Print the results
print(f"Exact Matches: {exact_matches}")
print(f"Mismatches: {mismatches}")

Exact Matches: 40
Mismatches: 217


In [ ]:
# 보조 지표 계산
from jiwer import wer as jiwer_wer, cer as jiwer_cer
jiwer_score = jiwer_wer(references, predictions) * 100
cer_score = jiwer_cer(references, predictions) * 100

print(f"JIWER WER: {jiwer_score:.2f}%, JIWER CER: {cer_score:.2f}%")

JIWER WER: 51.34%, JIWER CER: 30.91%


### Non-Dysarthria test

In [ ]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

# 커스텀 WER 계산 함수
def custom_wer(reference, hypothesis):
    """
    긴 예측 오류 또는 빈 예측 오류를 WER=1로 설정.
    """
    if not hypothesis or len(hypothesis.split()) > 2 * len(reference.split()):
        return 1.0  # WER = 1로 처리
    else:
        return metric.compute(predictions=[hypothesis], references=[reference])

# 데이터 로더 및 설정
non_dysarthria_data = DataLoader(data["non_dysarthria"], batch_size=8, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
normalizer = BasicTextNormalizer()

# 평가 결과 저장 변수
predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(non_dysarthria_data)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            # 예측 생성
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            # 라벨 처리
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

            # 결과 저장
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])

        # 메모리 정리
        del generated_tokens, labels, batch
    gc.collect()

# 커스텀 WER 계산
wer_values = [custom_wer(ref, pred) for ref, pred in zip(references, predictions)]
normalized_wer_values = [custom_wer(ref, pred) for ref, pred in zip(normalized_references, normalized_predictions)]

# 평균 WER 계산
wer = 100 * np.mean(wer_values)
normalized_wer = 100 * np.mean(normalized_wer_values)

eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

# 결과 출력
print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

  0%|          | 0/33 [00:00<?, ?it/s]<ipython-input-27-7f0fbc79f58c>:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 33/33 [00:56<00:00,  1.70s/it]


wer=71.58640322064447 and normalized_wer=20.727592770394327
{'eval/wer': 71.58640322064447, 'eval/normalized_wer': 20.727592770394327}


In [ ]:
for idx, (pred, ref) in enumerate(zip(predictions, references)):
    print(f"Index {idx}:\nPrediction: {pred}\nReference: {ref}\n")

Index 0:
Prediction:  Eat your reasons outdoors on the porch steps.
Reference: Eat your raisins outdoors on the porch steps.

Index 1:
Prediction:  He can't abide mistakes in his own game.
Reference: He can't abide mistakes in his own game.

Index 2:
Prediction:  Only two seemed really appealing to me.
Reference: Only two seemed really appealing to me.

Index 3:
Prediction:  Fold.
Reference: fold

Index 4:
Prediction:  Panel
Reference: panel

Index 5:
Prediction:  Duck.
Reference: duck

Index 6:
Prediction:  Low.
Reference: low

Index 7:
Prediction:  So he hides the manes.
Reference: So he hides the mayonnaise.

Index 8:
Prediction:  Preach
Reference: preach

Index 9:
Prediction:  beat
Reference: beat

Index 10:
Prediction:  We must be ready for any needed sacrifice.
Reference: We must be ready for any needed sacrifice.

Index 11:
Prediction:  Charm.
Reference: charm

Index 12:
Prediction:  Pono
Reference: funnel

Index 13:
Prediction:  It's impossible to deal with bureaucracy.
Referen

In [ ]:
def compare_lists(predictions, references):
    # Initialize counters
    exact_matches = 0
    mismatches = 0

    for pred, ref in zip(predictions, references):
        # Remove spaces for comparison
        pred_clean = pred.replace(" ", "")
        ref_clean = ref.replace(" ", "")

        if pred_clean == ref_clean:
            exact_matches += 1
        else:
            mismatches += 1

    return exact_matches, mismatches


# Compare the lists and get results
exact_matches, mismatches = compare_lists(predictions, references)

# Print the results
print(f"Exact Matches: {exact_matches}")
print(f"Mismatches: {mismatches}")

Exact Matches: 60
Mismatches: 197


In [ ]:
# 보조 지표 계산
from jiwer import wer as jiwer_wer, cer as jiwer_cer
jiwer_score = jiwer_wer(references, predictions) * 100
cer_score = jiwer_cer(references, predictions) * 100

print(f"JIWER WER: {jiwer_score:.2f}%, JIWER CER: {cer_score:.2f}%")

JIWER WER: 35.48%, JIWER CER: 14.79%
